# StressModels

In this notebook we give a quick start guide to StressModels in Pastas. Pastas uses StressModels to translate a hydrological stress to a contribution to a observed head time series. There are different stressmodels for unique use-cases. In Pastas we have the following StressModels:

In [ ]:
import pastas as ps

ps.stressmodels.__all__

## StressModel Attributes
Each stressmodel has the following attributes:

- stress (pandas Series)
- name (str, must be unique within a Model)
- rfunc (Pastas response function instance)

These attributes are optional but can be parsed:

- settings (dict or str, recommended)
- metadata (dict, optional)
- up (bool, indication whether contribution to the head is positive (`up=True`) or negative (`up=False`), given that stress is positive)
- gain_scale_factor (float, inital value estimator for the gain based on stress)

And these settings are infered based on the stress / settings:

- tmin (Timestamp) 
- tmax (Timestamp)
- freq (str, frequency)
- parameters (pandas DataFrame)


## Stresses
Lets read some stresses and visualize them:

In [ ]:
import pandas as pd

head = pd.read_csv(
    "../examples/data/head_nb1.csv", index_col=0, parse_dates=True
).squeeze()
prec = pd.read_csv(
    "../examples/data/rain_nb1.csv", index_col=0, parse_dates=True
).squeeze()
evap = pd.read_csv(
    "../examples/data/evap_nb1.csv", index_col=0, parse_dates=True
).squeeze()

# quick visualization
ps.plots.series(head=head, stresses=[prec, evap], hist=False);

__Figure 1: Observed head and stresses, in this case precipitation and evaporation (both positive)__

## Creating a StressModel

In [ ]:
sm = ps.StressModel(
    stress=prec,
    rfunc=ps.Gamma(),
    name="precipitation",
    settings="prec",  #
    metadata={"station": "location_x"},  # default is None
    up=True,  # default (head goes up if it rains)
    gain_scale_factor=None,  # default, retrieved from stress
)

### Stressmodel settings

Parsing the settings based with a string is equivalent to getting them from the rcParams which contains some build-in logic for up- and downsampling and filling missing data.

In [ ]:
settings = ps.rcParams["timeseries"]["prec"]
settings

Since evap is a positive series (>0), up has to be false because evaporation makes the head go down. Adding `stess=-evap` and using `up = True` would yield the same result in the simulation.

In [ ]:
sm2 = ps.StressModel(
    stress=evap,
    rfunc=ps.Gamma(),
    name="evaporation",
    settings="evap",  #
    metadata={"station": "location_x"},  # default is None
    up=False,
    gain_scale_factor=None,  # default, retrieved from stress
)

The RechargeModel combines the precipitation and evaporation series giving them the same response in a linear / nonlinear recharge combination.

In [ ]:
rm = ps.RechargeModel(
    prec=prec,
    evap=evap,
    rfunc=ps.Exponential(),  # default
    name="recharge",  # default
    recharge=ps.rch.Linear(),  # default, other options are ps.rch.FlexModel(), ps.rch.Peterson(), ps.rch.Berendrecht()
    settings=("prec", "evap"),  # default
)

Stressmodels are added to a Model using the `.add_stressmodel()` method and can be parsed seperately or as a list of stressmodels.

In [ ]:
ml = ps.Model(head)
ml.add_stressmodel(rm)
ml.add_stressmodel([sm, sm2])

### Other usecases for stressmodels

There are many other usecases for the different StressModels in Pastas. For each usecase there is an example such as:

- adding surface water levels / rivers
- adding (multiple) pumping wells
- adding trends
- changing responses

All these more specific examples with their settings can be found in the examples on [readthedocs](https://pastas.readthedocs.io/en/latest/examples/index.html)